In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import re
# Load the questionnaire data
questionnaire_df = pd.read_excel('data/questionnaire_data-561422-2025-11-17-1240.xlsx')

/home/michaelt/.cache/pypoetry/virtualenvs/in9060-as2-Pf7P2CoS-py3.12/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
# Rename questionnaire_df columns to concise snake_case with condition suffixes


# Mapping from full question text to short code
_base_map = {
    'Participant number': 'participant_id',
    'What is your gender': 'gender',
    'How old are you?': 'age',
    'What is your dominant hand?': 'dominant_hand',
    'How experienced are you with robotic systems?': 'robotics_experience',
    'Did you experience delays between your actions and the robot\'s movements?': 'delay',
    'How difficult was it to perform the task?': 'difficulty',
    'I felt like I was controlling the movement of the robot': 'control',
    'It felt like the robot was part of my body': 'body'
}

repeating_keys = {'delay','difficulty','control','body'}
rename_map = {}
for col in questionnaire_df.columns:
    if col.startswith('$'):  # keep metadata columns unchanged
        continue
    # Remove HTML entity artifacts
    col_clean = col.replace('&#39;', "'")
    # Extract suffix like .1, .2, etc.
    m = re.search(r'\.(\d+)$', col_clean)
    base = re.sub(r'\.(\d+)$', '', col_clean)
    short = _base_map.get(base, None)
    if short is None:
        # If unknown and not metadata, create a generic snake_case key
        generic = re.sub(r'[^A-Za-z0-9]+', '_', base).strip('_').lower()
        short = generic or base.lower()
    if m:
        suffix = f"_{m.group(1)}"  # condition index from original numbering
    elif short in repeating_keys:
        suffix = "_0"  # first occurrence without explicit suffix
    else:
        suffix = ""
    rename_map[col] = short + suffix

questionnaire_df = questionnaire_df.rename(columns=rename_map)

print('Renamed columns (first 25):')
print(list(questionnaire_df.columns)[:25])

# Filter out columns containing "$"
filtered_columns = [col for col in questionnaire_df.columns if '$' not in col and col != 'Participant number']
print('Filtered columns (first 25):')
print(filtered_columns[:25])

participant_data_columns = filtered_columns[0:4]
task_one_columns = filtered_columns[5:25]
task_two_columns = filtered_columns[25:]

participant_df = questionnaire_df[participant_data_columns]
task_one_df = questionnaire_df[task_one_columns]
task_two_df = questionnaire_df[task_two_columns]
all_tasks_df = task_one_df + task_two_df

print('Filtered columns (first 25):')
print(list(task_one_df.columns)[:25])

metric_mask_delay = [col for col in all_tasks_df.columns if col.startswith('delay_')]
metric_mask_difficulty = [col for col in all_tasks_df.columns if col.startswith('difficulty_')]
metric_mask_control = [col for col in all_tasks_df.columns if col.startswith('control_')]
metric_mask_body = [col for col in all_tasks_df.columns if col.startswith('body_')]

# Filter task_one_df for male participants
male_mask = participant_df['gender'] == 'Male'
female_mask = participant_df['gender'] == 'Female'
Other_mask = participant_df['gender'] == 'Other'
task_one_male_df = task_one_df[male_mask]
task_one_female_df = task_one_df[female_mask]



Renamed columns (first 25):
['$submission_id', '$created', 'participant_id', 'gender', 'age', 'dominant_hand', 'robotics_experience', 'delay_0', 'difficulty_0', 'control_0', 'body_0', 'delay_1', 'difficulty_1', 'control_1', 'body_1', 'delay_2', 'difficulty_2', 'control_2', 'body_2', 'delay_3', 'difficulty_3', 'control_3', 'body_3', 'delay_4', 'difficulty_4']
Filtered columns (first 25):
['participant_id', 'gender', 'age', 'dominant_hand', 'robotics_experience', 'delay_0', 'difficulty_0', 'control_0', 'body_0', 'delay_1', 'difficulty_1', 'control_1', 'body_1', 'delay_2', 'difficulty_2', 'control_2', 'body_2', 'delay_3', 'difficulty_3', 'control_3', 'body_3', 'delay_4', 'difficulty_4', 'control_4', 'body_4']
Filtered columns (first 25):
['delay_0', 'difficulty_0', 'control_0', 'body_0', 'delay_1', 'difficulty_1', 'control_1', 'body_1', 'delay_2', 'difficulty_2', 'control_2', 'body_2', 'delay_3', 'difficulty_3', 'control_3', 'body_3', 'delay_4', 'difficulty_4', 'control_4', 'body_4']


In [ ]:
# In theory this is all I need
task_one_difficulty_df = task_one_df[[col for col in task_one_df.columns if col.startswith('difficulty_')]]
task_one_difficulty_df_male = task_one_difficulty_df[male_mask]
task_one_difficulty_df_female = task_one_difficulty_df[female_mask]

task_one_stats_male = pd.DataFrame({
    'Mean': task_one_difficulty_df_male.mean(numeric_only=True),
    'Variance': task_one_difficulty_df_male.var(numeric_only=True)
})
task_one_stats_female = pd.DataFrame({
    'Mean': task_one_difficulty_df_female.mean(numeric_only=True),
    'Variance': task_one_difficulty_df_female.var(numeric_only=True)
})

print("Task One Statistics Male:")
print(task_one_stats_male)
print("\nTask One Statistics Female:")
print(task_one_stats_female)

Task One Statistics Male:
                  Mean  Variance
difficulty_0  2.958333  0.737319
difficulty_1  2.333333  1.014493
difficulty_2  2.416667  0.775362
difficulty_3  2.333333  1.362319
difficulty_4  2.458333  1.128623

Task One Statistics Female:
              Mean  Variance
difficulty_0   2.8  0.844444
difficulty_1   2.6  0.933333
difficulty_2   2.7  0.900000
difficulty_3   2.4  0.488889
difficulty_4   2.6  0.488889


In [19]:
# Example 3: Comprehensive statistics with all variance types
# Calculate and display statistics in a table
import fig.variance_plot as vp
# importlib.reload(vp)
all_variance_stats = vp.calculate_statistics(
    distributions=[
        task_one_difficulty_df_male, task_one_difficulty_df_female
    ],
    group_labels=['Male', 'Female'],
    # variance_types=['sample', 'population', 'std', 'mse', 'ss', 'tukey', 'sst', 'sse', 'ssa', 'ftest'],
    variance_types=['sample', 'mse', 'ss', 'tukey', 'sst', 'sse', 'ssa', 'ftest'],
    output_csv='comprehensive_stats.csv'
)

print("\nComprehensive Statistics - All Variance Types")
print("=" * 140)
print(all_variance_stats.to_string(index=False))

print("\n\nColumn Descriptions:")
print("-" * 140)
print("N         : Sample size")
print("Mean      : Arithmetic mean")
print("SAMPLE    : Sample variance (ddof=1, unbiased estimator)")
print("POPULATION: Population variance (ddof=0)")
print("STD       : Standard deviation (ddof=1)")
print("MSE       : Mean Squared Error from mean")
print("SS        : Sum of Squares (within group)")
print("TUKEY     : Tukey's HSD variance estimate (variance/n)")
print("SST       : Total Sum of Squares (from grand mean)")
print("SSE       : Error/Within Sum of Squares (from group means)")
print("SSA       : Among/Between Groups Sum of Squares")
print("FTEST     : F-test component (within-group variance)")


Statistics saved to: comprehensive_stats.csv

Comprehensive Statistics - All Variance Types
  Group   N     Mean   SAMPLE      MSE         SS    TUKEY        SST        SSE      SSA    FTEST
   Male 120 2.500000 1.025210 1.016667 122.000000 0.008543 122.149481 122.000000 0.149481 1.025210
 Female  50 2.620000 0.689388 0.675600  33.780000 0.013788  34.138754  33.780000 0.358754 0.689388
Overall 170 2.535294 0.924782 0.919343 156.288235 0.005440 156.288235 156.288235 0.000000 0.924782


Column Descriptions:
--------------------------------------------------------------------------------------------------------------------------------------------
N         : Sample size
Mean      : Arithmetic mean
SAMPLE    : Sample variance (ddof=1, unbiased estimator)
POPULATION: Population variance (ddof=0)
STD       : Standard deviation (ddof=1)
MSE       : Mean Squared Error from mean
SS        : Sum of Squares (within group)
TUKEY     : Tukey's HSD variance estimate (variance/n)
SST       : Total Su